In [72]:
import boto3
import sagemaker
import os
import numpy as np

In [50]:
# session and role
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# location of stored sata
data_dir = 'modelling'
# S3 location
bucket = sagemaker_session.default_bucket()
# set prefix, a descriptive name for a directory  
prefix = 'sagemaker/audio_emotion_detection'

## Load and deploy saved model

An AWS blogpost about deployment of a keras model on SageMaker (https://aws.amazon.com/blogs/machine-learning/deploy-trained-keras-or-tensorflow-models-using-amazon-sagemaker/) gave me a good general idea how to proceed. First, I changed the training approach to save model in a format enabling deployment on SageMaker. The blogpost described a workaround to deploy a model, by using sagemaker.tensorflow.model.TensorFlowModel and simulating its creation with an empty train.py file. As I discovered from the Sagemaker documentation, there is now a better way to deploy directly from model artifacts (https://sagemaker.readthedocs.io/en/stable/using_tf.html#deploying-directly-from-model-artifacts).

In [14]:
!ls export/1

saved_model.pb	variables


## Deploy the trained model

TensorFlow Serving Model
https://sagemaker.readthedocs.io/en/stable/sagemaker.tensorflow.html

Specification of *framework_version='2.0.0'* alighns the TensorFlow version of the trained model with the version of the serving model. Otherwise, an error was thrown.


In [51]:
from sagemaker.tensorflow.serving import Model
model = Model(model_data = 's3://' + bucket + '/' + prefix + '/' + 'model.tar.gz', 
              role = role,
             framework_version='2.0.0')

In [52]:
%%time
predictor = model.deploy(initial_instance_count=1, 
                         instance_type='ml.m4.xlarge')

--------------------------------------------------------------------------------------!CPU times: user 645 ms, sys: 22 ms, total: 667 ms
Wall time: 7min 14s


In [61]:
predictor.endpoint

'tensorflow-inference-2019-12-26-08-24-13-228'

## Making predictions

Sending a preprocessed sound data (MFCC feature) to the endpoint results in probability prediction for each of the classes. The endpoint accepts different formats and an appropriate should be chosen to be used with the lambda function.

JSON-formats:

In [53]:
mfcc = [-5.5037286e+02,  4.8238766e+01,  1.8442627e+01,  6.5049920e+00,
        8.2730311e-01, -6.1545024e+00, -2.4476097e+01, -7.4760280e+00,
       -1.1998719e+01, -9.4280348e+00, -7.5956612e+00, -5.5952277e+00,
       -4.4645872e+00, -6.2611837e+00, -3.2972498e+00, -8.0563803e+00,
       -4.3905830e+00, -2.1417410e+00, -6.5392509e+00, -6.4882522e+00,
       -6.0786166e+00, -8.1123333e+00, -5.5041442e+00,  4.6315003e-02,
       -1.7572067e+00,  8.9478511e-01, -1.2644883e-01,  2.5937853e+00,
       -1.1436307e+00,  3.4961894e+00,  2.2928705e+00,  3.7813089e+00,
        4.3201895e+00,  1.7073919e+00,  1.4069341e+00, -1.3996479e-01,
       -3.3032024e-01,  1.0882645e+00, -5.8361721e-01,  1.6979592e+00]

In [54]:
input = {
  'instances': [mfcc]
}

In [55]:
result = predictor.predict(input)
result

{'predictions': [[0.0174345262,
   0.0509620309,
   0.61150676,
   0.00767038902,
   0.312426269]]}

In [56]:
input = [mfcc]

In [57]:
result = predictor.predict(input)
result

{'predictions': [[0.0174345262,
   0.0509620309,
   0.61150676,
   0.00767038902,
   0.312426269]]}

CSV (comma-separated values) fortmat:

In [75]:
from sagemaker.tensorflow.serving import Predictor
# create a Predictor with JSON serialization
predictor = Predictor(predictor.endpoint, serializer=sagemaker.predictor.csv_serializer)

# CSV-formatted string input
input = mfcc

result = predictor.predict(input)
result

{'predictions': [[0.0174345262,
   0.0509620309,
   0.61150676,
   0.00767038902,
   0.312426269]]}

In [76]:
result = result['predictions'][0]

Transform into a label prediction:

In [70]:
class_mapping_dict = {'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}

In [77]:
# get the index of a maximum value
max_idx = np.argmax(result)
# get key of a value equal to the index of a maximum value
# sourced from https://stackoverflow.com/questions/8023306/get-key-by-value-in-dictionary
result_label = list(class_mapping_dict.keys())[list(class_mapping_dict.values()).index(max_idx)]
result_label

'happy'

## Invoke endpoint as by a Lambda function

In [111]:
input = {'instances': [mfcc]}

In [112]:
import json
json.dumps(input)

'{"instances": [[-550.37286, 48.238766, 18.442627, 6.504992, 0.82730311, -6.1545024, -24.476097, -7.476028, -11.998719, -9.4280348, -7.5956612, -5.5952277, -4.4645872, -6.2611837, -3.2972498, -8.0563803, -4.390583, -2.141741, -6.5392509, -6.4882522, -6.0786166, -8.1123333, -5.5041442, 0.046315003, -1.7572067, 0.89478511, -0.12644883, 2.5937853, -1.1436307, 3.4961894, 2.2928705, 3.7813089, 4.3201895, 1.7073919, 1.4069341, -0.13996479, -0.33032024, 1.0882645, -0.58361721, 1.6979592]]}'

In [113]:
import boto3

runtime = boto3.Session().client('sagemaker-runtime')

In [128]:
# Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
response = runtime.invoke_endpoint(EndpointName = predictor.endpoint, # The name of the endpoint
                                       ContentType = 'application/json', # The data format that is expected
                                       Body = json.dumps(input) # The preprocessed audio file
                                  )

In [139]:
response

{'ResponseMetadata': {'RequestId': '3f9b9ef1-1fa8-4191-a7c5-a50f50735380',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '3f9b9ef1-1fa8-4191-a7c5-a50f50735380',
   'x-amzn-invoked-production-variant': 'AllTraffic',
   'date': 'Thu, 26 Dec 2019 10:27:52 GMT',
   'content-type': 'application/json',
   'content-length': '98'},
  'RetryAttempts': 0},
 'ContentType': 'application/json',
 'InvokedProductionVariant': 'AllTraffic',
 'Body': <botocore.response.StreamingBody at 0x7f9b99821240>}

In [130]:
# The response is an HTTP response whose body contains the result of our inference
result = response['Body'].read().decode('utf-8')
result

'{\n    "predictions": [[0.0174345262, 0.0509620309, 0.61150676, 0.00767038902, 0.312426269]\n    ]\n}'

In [136]:
# thansform to json
result = json.loads(result)

In [137]:
# extract an array of probability predictions
result = result['predictions'][0]
    
# get the index of a maximum value
max_idx = np.argmax(result)
# get key of a value equal to the index of a maximum value
# sourced from https://stackoverflow.com/questions/8023306/get-key-by-value-in-dictionary
result_label = list(class_mapping_dict.keys())[list(class_mapping_dict.values()).index(max_idx)]

In [138]:
result_label

'happy'

In [ ]:
# Accepts a predictor endpoint as input
# And deletes the endpoint by name
def delete_endpoint(predictor):
        try:
            boto3.client('sagemaker').delete_endpoint(EndpointName=predictor.endpoint)
            print('Deleted {}'.format(predictor.endpoint))
        except:
            print('Already deleted: {}'.format(predictor.endpoint))
